<a href="https://colab.research.google.com/github/Rohan-pashikanti/NLP_batch_05/blob/main/Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]

(i) Build the Transformer Model on above dataset

(ii) Train the model using 20, 60, 70 epochs

 (iii) After training, use the model to generate new text by feeding it an initial seed text

(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
# Example text data
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

# Tokenization and preparing sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for i in range(1, len(tokenizer.texts_to_sequences([text])[0])):
    n_gram_sequence = tokenizer.texts_to_sequences([text])[0][:i+1]
    input_sequences.append(n_gram_sequence)

# Pad sequences
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

# Inputs and labels
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Define Transformer-based model using MultiHeadAttention
def transformer_block(vocab_size, seq_len):
    inputs = tf.keras.layers.Input(shape=(seq_len,))
    embedding = tf.keras.layers.Embedding(vocab_size, 64)(inputs)

    # Multi-Head Attention Layer
    attention_output = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=64)(embedding, embedding)
    attention_output = tf.keras.layers.LayerNormalization()(attention_output + embedding)

    # Feed-forward network
    ff_output = tf.keras.layers.Dense(128, activation='relu')(attention_output)
    ff_output = tf.keras.layers.Dense(64)(ff_output)
    ff_output = tf.keras.layers.LayerNormalization()(ff_output + attention_output)

    outputs = tf.keras.layers.GlobalAveragePooling1D()(ff_output)
    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(outputs)

    model = tf.keras.models.Model(inputs, outputs)
    return model

(i) Build the Transformer Model on above dataset

In [3]:
# Build model
model = transformer_block(total_words, max_seq_len-1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 55)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 55, 64)         │          2,624 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 55, 64)         │         33,216 │ embedding[0][0],       │
│ (MultiHeadAttention)      │                        │                │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 55, 64)         │              0 │ multi_head_attention[… │
│                           │                        │                │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 55, 64)         │            128 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 55, 128)        │          8,320 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 55, 64)         │          8,256 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 55, 64)         │              0 │ dense_1[0][0],         │
│                           │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 55, 64)         │            128 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 64)             │              0 │ layer_normalization_1… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 41)             │          2,665 │ global_average_poolin… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 55,337 (216.16 KB)

 Trainable params: 55,337 (216.16 KB)

 Non-trainable params: 0 (0.00 B)

(ii) Train the model using 20, 60, 70 epochs

In [4]:
# Train the model (20, 60, 70 epochs)
epochs_list = [20, 60, 70]
history = []
for epoch in epochs_list:
    print(f"Training with {epoch} epochs...")
    h = model.fit(X, y, epochs=epoch, verbose=1)
    history.append(h)

Training with 20 epochs...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.0347 - loss: 3.8708
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.0831 - loss: 3.6111
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0831 - loss: 3.5868
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.1248 - loss: 3.3833
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1473 - loss: 3.3989
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.1057 - loss: 3.3896 
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1369 - loss: 3.2909
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1161 - loss: 3.3073
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.1265 - loss: 3.2854
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1924 - loss: 3.2250
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.1369 - loss: 3.2010
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accur

(iii) After training, use the model to generate new text by feeding it an initial seed text

(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.

In [5]:
# Text generation using seed text
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=1)
        output_word = tokenizer.index_word[predicted_word_index[0]]
        seed_text += " " + output_word
    return seed_text

seed_text = "Once upon a time"
generated_text = generate_text(seed_text, 50, model, max_seq_len)
print("Generated text:", generated_text)

Generated text: Once upon a time a a girl named red riding hood she loved her who who lived the the the woods one day asked her mother her mother take a a of her of to to her way way woods a through the woods met met a big bad bad wolf eat to to
